In [ ]:
import sys
from tqdm import tqdm as tqdm

In [2]:
!pip install anndata
!pip install scanpy

In [2]:
import pandas as pd
import anndata as ad
import numpy as np
from scipy import sparse
from tqdm import tqdm
import scanpy as sc
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('data/counts_pat12.csv', index_col=0)
df['subclone'] = df['subclone'].replace(['healthy.healthy.1.1.1', 'healthy.healthy.1.1.2', 'healthy.healthy.1.2'], 'healthy')

In [ ]:
df['subclone'].unique()

In [8]:
df

,22848,14,10058,5825,55347,25864,51099,57406,10152,25890,...,125893,55769,7644,81931,8233,65982,79149,90204,158586,subclone
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGTCAGAGC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,all_observations.all_observations.1.1.1.1
AAACCCACAGCGTGAA-1,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,1,all_observations.all_observations.1.1.1.1
AAACCCAGTGCAGGAT-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,all_observations.all_observations.1.1.1.1
AAACGAAAGTGGAATT-1,1,0,0,1,0,0,4,0,0,0,...,0,0,1,0,0,0,0,0,0,all_observations.all_observations.1.1.1.1
AAACGAACAACTCGAT-1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,all_observations.all_observations.1.1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGCAAAGGGCT-1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,all_observations.all_observations.1.1.1.2
TTTGTTGCAACCGTGC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,all_observations.all_observations.1.1.1.2
TTTGTTGCATTAGGAA-1,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,all_observations.all_observations.1.1.1.2


In [9]:
np.random.seed(42)

In [10]:
import pandas as pd
import numpy as np
from scipy import sparse

def sum_by_df(df: pd.DataFrame, group_col: str) -> pd.DataFrame:
    """
    Aggregates gene expression counts by group within a single DataFrame.

    Parameters:
    - df: pandas DataFrame. Each row is a cell. 
          Columns are gene expression (numeric), plus one column with group labels.
    - group_col: string. Name of the column that contains group/subclone labels.

    Returns:
    - A DataFrame indexed by group_col, where each row is a group (e.g., subclone) 
      and columns are the summed gene expression features.
    """

    # Ensure group_col exists
    if group_col not in df.columns:
        raise ValueError(f"Group column '{group_col}' not found in dataframe columns")

    # List all gene feature columns (everything except the group column)
    gene_cols = [col for col in df.columns if col != group_col]

    # Ensure group_col is categorical
    df[group_col] = df[group_col].astype('category')
    cat = df[group_col].cat
    n_obs = df.shape[0]

    # Build a sparse indicator matrix for efficient summing
    indicator = sparse.coo_matrix(
        (
            np.ones(n_obs, dtype=bool),
            (cat.codes, np.arange(n_obs))
        ),
        shape=(len(cat.categories), n_obs)
    )

    # Extract and validate gene expression matrix (cells x genes)
    data_matrix = df[gene_cols].values
    if not sparse.issparse(data_matrix):
        data_matrix = sparse.csr_matrix(data_matrix)

    # Matrix multiplication: (groups x cells) @ (cells x genes) = (groups x genes)
    summed = indicator @ data_matrix

    # Output DataFrame, indexed by group, with gene columns
    summed_df = pd.DataFrame(
        summed.toarray(),
        index=cat.categories,
        columns=gene_cols
    )
    summed_df.index.name = group_col

    return summed_df

In [11]:
df['subclone'].unique()

array(['all_observations.all_observations.1.1.1.1',
       'all_observations.all_observations.1.1.1.2',
       'all_observations.all_observations.1.1.2.1',
       'all_observations.all_observations.1.1.2.2',
       'all_observations.all_observations.1.2.1.1',
       'all_observations.all_observations.1.2.1.2',
       'all_observations.all_observations.1.2.2.1',
       'all_observations.all_observations.1.2.2.2', 'healthy'],
      dtype=object)

In [ ]:
for i in tqdm(range(100)):
    bootstrap_ids = np.random.choice(df.index, size=4000)
    subsampled_df = df.loc[bootstrap_ids]
    bulked = sum_by_df(subsampled_df, 'subclone')
    bulked.to_csv(f'data/pseudobulks/bulked_pat12_iter{i+1}.csv', index=True)

100%|██████████| 100/100 [01:38<00:00,  1.02it/s]


In [ ]:
# Define path template for input files
path_template = 'data/pseudobulks/bulked_pat12_iter{}.csv'

# Load the first bulk
i = 0
bulk_df = pd.read_csv(path_template.format(i+1))
bulk_df = bulk_df.reset_index(drop=False)
bulk_df['bulk'] = f'bulk_{i+1}'
bulk_df['bulk_id'] = bulk_df['bulk'] + '_id_' + bulk_df['index'].astype(str)
bulk_df.set_index('bulk_id', inplace=True)

# Iterate through subsequent bulks and concatenate
for i in tqdm(range(0, 100)):
    df_ = pd.read_csv(path_template.format(i+1))
    df_ = df_.reset_index(drop=False)
    df_['bulk'] = f'bulk_{i+1}'
    df_['bulk_id'] = df_['bulk'] + '_id_' + df_['index'].astype(str)
    df_.set_index('bulk_id', inplace=True)
    bulk_df = pd.concat([bulk_df, df_], axis=0, join='outer')


100%|██████████| 100/100 [00:14<00:00,  6.83it/s]


In [14]:
bulk_df['subclone_bulk'] = (bulk_df['subclone'].astype(str) + '_' + bulk_df['bulk'].astype(str)).astype('category')

In [15]:
# Select all numeric columns except those not to aggregate (like old index)
numeric_cols = bulk_df.select_dtypes(include=['number']).columns.tolist()
if 'index' in numeric_cols:
    numeric_cols.remove('index')

# Group and compute mean for all numeric columns per subclone_bulk
agg_df = bulk_df.groupby('subclone_bulk')[numeric_cols].mean().reset_index()

/var/folders/rk/_8vnc6913c5fyc733rjczgp80000gn/T/ipykernel_813/4061024358.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg_df = bulk_df.groupby('subclone_bulk')[numeric_cols].mean().reset_index()


In [16]:
# List all gene columns (numeric data only)
gene_cols = [col for col in agg_df.columns if col not in ['subclone_bulk']]

# Library size normalization: scale each row to sum to 1e4 (like sc.pp.normalize_total)
agg_df[gene_cols] = (
    agg_df[gene_cols].div(agg_df[gene_cols].sum(axis=1), axis=0) * 1e4
)

# Log-transform (base 2), similar to sc.pp.log1p
agg_df[gene_cols] = np.log2(agg_df[gene_cols] + 1)


In [17]:
def robust_z_scores(X):
    median = np.median(X, axis=0)
    diff = X - median
    MAD = np.median(np.abs(diff), axis=0)
    z = diff / (MAD * 1.4826 + 1e-8)  # Add epsilon to avoid divide-by-zero
    return z

z = robust_z_scores(agg_df[gene_cols].values)
agg_df[gene_cols] = z


In [18]:
agg_df['subclone'] = agg_df['subclone_bulk'].astype(str).str.split('_bulk').str[0]

/var/folders/rk/_8vnc6913c5fyc733rjczgp80000gn/T/ipykernel_813/1883174851.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  agg_df['subclone'] = agg_df['subclone_bulk'].astype(str).str.split('_bulk').str[0]


In [19]:
agg_df

,subclone_bulk,22848,14,10058,5825,55347,25864,51099,57406,10152,...,125893,55769,7644,81931,8233,65982,79149,90204,158586,subclone
0,all_observations.all_observations.1.1.1.1_bulk_1,0.012586,-0.341983,0.177615,0.425842,0.272929,-0.118279,0.608392,0.155371,-0.516581,...,0.192490,0.034718,0.245619,0.037794,1.375235,0.228096,-1.061394,0.713546,0.207286,all_observations.all_observations.1.1.1.1
1,all_observations.all_observations.1.1.1.1_bulk_10,0.469749,0.613245,0.358384,0.862491,0.057994,0.432841,0.773765,0.375148,-0.063584,...,0.696232,0.643414,0.673900,0.277425,0.870757,-0.024309,0.522661,0.849933,-0.569609,all_observations.all_observations.1.1.1.1
2,all_observations.all_observations.1.1.1.1_bulk...,0.472176,-0.808622,0.290305,0.280894,0.191449,0.618453,-0.308245,0.554883,-0.523922,...,-0.047755,1.107683,0.091167,0.515115,1.303456,0.369311,0.429700,0.578950,-0.520660,all_observations.all_observations.1.1.1.1
3,all_observations.all_observations.1.1.1.1_bulk_11,0.846660,-0.745931,1.389426,0.346665,0.097176,1.022787,-0.120233,0.649446,-0.957665,...,0.514422,0.921392,0.128027,0.741372,0.765246,0.315727,0.920082,1.222085,0.437020,all_observations.all_observations.1.1.1.1
4,all_observations.all_observations.1.1.1.1_bulk_12,-0.385670,-0.446423,0.795631,0.168129,-0.346513,1.268805,-0.344068,-0.252287,0.206239,...,-0.560890,-0.057738,0.254557,-0.542796,0.752571,-0.081409,0.112673,0.146789,0.065532,all_observations.all_observations.1.1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,healthy_bulk_95,0.325054,-0.044033,-1.625449,0.513177,2.249376,-3.073757,2.421623,-0.831729,3.056865,...,-1.154979,-3.115316,1.053661,0.127281,-4.614800,3.017592,16.926806,4.466780,-2.830354,healthy
896,healthy_bulk_96,0.501786,2.427980,-1.625449,-2.294321,2.848484,-3.073757,2.485434,-0.831729,0.827817,...,-1.154979,-1.096780,3.032937,0.989436,-3.401546,-3.940815,12.451246,-1.489708,-2.830354,healthy
897,healthy_bulk_97,-2.976292,-1.283131,-1.625449,-4.057839,-0.935828,-3.073757,7.888504,-0.831729,-2.691269,...,-1.154979,-4.205430,2.398919,-1.731884,-3.132807,-1.973523,1.645050,-1.489708,1.162513,healthy
898,healthy_bulk_98,-0.585619,-1.426472,-1.625449,-4.057839,-1.695463,-3.073757,1.555092,-0.831729,0.114688,...,-1.154979,1.128744,6.401550,5.241976,-4.614800,-1.126143,3.363671,2.481830,-2.830354,healthy


In [20]:
df_final_expression = (
    agg_df
    .groupby('subclone')[gene_cols]
    .mean()
    .reset_index()
)

In [21]:
df_final_expression

,subclone,22848,14,10058,5825,55347,25864,51099,57406,10152,...,730051,125893,55769,7644,81931,8233,65982,79149,90204,158586
0,all_observations.all_observations.1.1.1.1,0.148140,-0.229601,0.296848,0.409044,0.071311,0.562366,0.121293,0.197433,-0.353666,...,0.198631,0.108130,0.816627,0.488041,0.233459,0.803303,0.344392,-0.025319,0.707017,-0.128328
1,all_observations.all_observations.1.1.1.2,0.593058,-0.454420,-0.240199,-0.082675,0.139807,-0.058565,0.566568,0.292385,-0.344798,...,-0.396157,0.272517,-0.050007,-0.227736,-0.016729,0.163020,-0.783741,0.071589,-0.445336,-0.419597
2,all_observations.all_observations.1.1.2.1,-0.705341,-2.038836,-0.353192,1.648505,0.498171,-0.695973,-0.331551,-0.003826,0.615553,...,-1.366182,0.806462,-0.266349,0.339302,-1.731884,0.643015,-0.156431,0.163253,-0.312975,0.333796
3,all_observations.all_observations.1.1.2.2,-0.460536,0.780426,4.176268,-0.352031,0.898114,-0.216305,1.747624,0.420956,1.399801,...,0.889565,1.766927,2.090503,0.265846,-0.824041,-1.490700,0.795083,-0.364079,-0.739236,-0.767422
4,all_observations.all_observations.1.2.1.1,-0.107394,0.827661,1.368211,0.727660,-1.342604,0.018018,-0.473066,-0.321727,0.628166,...,0.456149,-0.747851,-0.048359,-0.898078,-0.495087,0.659436,0.753503,-0.996208,-0.103964,0.167373
5,all_observations.all_observations.1.2.1.2,0.440698,0.058445,-0.215537,-0.834572,-0.227516,0.917001,0.015705,0.182812,-0.086653,...,0.417451,0.344027,-0.525748,0.155874,0.573280,0.163885,0.161197,0.282291,0.174076,0.429536
6,all_observations.all_observations.1.2.2.1,-1.109742,0.636460,0.115769,0.022301,0.327067,0.488474,-0.260936,0.824811,-0.204317,...,0.084825,0.255892,-0.040026,-1.023594,1.000747,-0.581642,-0.525568,-0.528855,0.108979,-0.025976
7,all_observations.all_observations.1.2.2.2,-0.147300,-0.053825,-0.234508,0.206008,0.189059,-0.200918,-1.134472,-0.466118,-0.155920,...,0.212237,-0.358834,-0.157698,-0.225525,0.268647,-0.228284,-0.436979,0.341126,0.668112,1.530188
8,healthy,-0.213993,-0.015561,-1.625449,-2.706548,-0.666572,-3.073757,2.080631,-0.831729,0.603299,...,0.289239,-1.154979,-1.796470,2.692196,0.435690,-3.191444,0.192569,8.068591,1.440540,-0.434516


In [ ]:
variants = pd.read_csv('data/aggregated_muts_12.csv')

In [23]:
values_dict = variants.iloc[0].to_dict()
df_final = df_final_expression.assign(**values_dict)

/var/folders/rk/_8vnc6913c5fyc733rjczgp80000gn/T/ipykernel_813/338592391.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final = df_final_expression.assign(**values_dict)
/var/folders/rk/_8vnc6913c5fyc733rjczgp80000gn/T/ipykernel_813/338592391.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final = df_final_expression.assign(**values_dict)
/var/folders/rk/_8vnc6913c5fyc733rjczgp80000gn/T/ipykernel_813/338592391.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

In [24]:
df_final

,subclone,22848,14,10058,5825,55347,25864,51099,57406,10152,...,ZSCAN18,ZSCAN2,ZSCAN32,ZSCAN5A,ZW10,ZWILCH,ZXDC,ZYX,ZZEF1,ZZZ3
0,all_observations.all_observations.1.1.1.1,0.148140,-0.229601,0.296848,0.409044,0.071311,0.562366,0.121293,0.197433,-0.353666,...,6,1,1,6,1,3,4,8,12,6
1,all_observations.all_observations.1.1.1.2,0.593058,-0.454420,-0.240199,-0.082675,0.139807,-0.058565,0.566568,0.292385,-0.344798,...,6,1,1,6,1,3,4,8,12,6
2,all_observations.all_observations.1.1.2.1,-0.705341,-2.038836,-0.353192,1.648505,0.498171,-0.695973,-0.331551,-0.003826,0.615553,...,6,1,1,6,1,3,4,8,12,6
3,all_observations.all_observations.1.1.2.2,-0.460536,0.780426,4.176268,-0.352031,0.898114,-0.216305,1.747624,0.420956,1.399801,...,6,1,1,6,1,3,4,8,12,6
4,all_observations.all_observations.1.2.1.1,-0.107394,0.827661,1.368211,0.727660,-1.342604,0.018018,-0.473066,-0.321727,0.628166,...,6,1,1,6,1,3,4,8,12,6
5,all_observations.all_observations.1.2.1.2,0.440698,0.058445,-0.215537,-0.834572,-0.227516,0.917001,0.015705,0.182812,-0.086653,...,6,1,1,6,1,3,4,8,12,6
6,all_observations.all_observations.1.2.2.1,-1.109742,0.636460,0.115769,0.022301,0.327067,0.488474,-0.260936,0.824811,-0.204317,...,6,1,1,6,1,3,4,8,12,6
7,all_observations.all_observations.1.2.2.2,-0.147300,-0.053825,-0.234508,0.206008,0.189059,-0.200918,-1.134472,-0.466118,-0.155920,...,6,1,1,6,1,3,4,8,12,6
8,healthy,-0.213993,-0.015561,-1.625449,-2.706548,-0.666572,-3.073757,2.080631,-0.831729,0.603299,...,6,1,1,6,1,3,4,8,12,6


Need to align to tahoe order

In [ ]:
df_final.to_csv('data/gene_expression_processed_pat12.csv', index=False)